### Import settings

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.0  pyspark-shell'

In [2]:
import findspark
findspark.init('/opt/spark')
from pyspark import SparkContext,SparkConf
sc = SparkContext()
#With Cluster
#conf = (SparkConf()
#         .setMaster("spark://10.200.5.39:7077")
#         .set("spark.driver.host","10.200.5.39") 
#         .set("spark.executor.memory","58g")
#         .set('spark.driver.memory', '60G')
#         .setAppName("Classification"))
#sc = SparkContext(conf=conf)

In [3]:
import pyspark
import binascii
from pyspark.sql import SQLContext
from functools import reduce
from sklearn import tree
from pyspark.sql.types import *
#import pygraphviz
import pyspark.sql.functions as f
#import networkx as nx
#from networkx.readwrite import json_graph
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn import datasets,metrics
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
import numpy as np
import json
#import tensorflow as tf
from IPython.display import Image
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from networkx.drawing.nx_pydot import write_dot
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score  
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,f1_score,accuracy_score,precision_score,recall_score,matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
import pickle
sqlContext = SQLContext(sc)

### Functions space

In [ ]:
path = "/Kriptosare.class"
path_directory="/analysis/analysis_rest/final"
modelpath='/Kriptosare.class/train_model'

In [4]:
cSchema  = StructType([
    StructField("user", StringType(), True),
    StructField("label", IntegerType(), True)
])

cSchema2  = StructType([
    StructField("user", StringType(), True),
    StructField("predict_class1", FloatType(), True),
    StructField("predict_class2", FloatType(), True),
    StructField("predict_class3", FloatType(), True),
    StructField("predict_class4", FloatType(), True),
    StructField("predict_class5", FloatType(), True),
    StructField("predict_class6", FloatType(), True)
])

def join_dataframe_intag(DF,pref):
    DF=DF.groupBy("user").agg(f.collect_list("label").alias("list"))
    DF = DF.select("user",f.explode("list").alias("value")).groupBy("user","value").agg(f.count("value").alias("cnt"))
    DF2 = DF.groupBy("user").agg(f.count("user").alias("cnt")).drop("cnt")
    DF2 = DF2.alias("a").join(DF.alias('b'),(f.col("a.user")==f.col("b.user")) & (f.col("b.value")==1),"leftouter")\
    .select("a.user","b.cnt").withColumnRenamed("cnt",pref+"_cnt1")
    DF2 = DF2.alias("a").join(DF.alias('b'),(f.col("a.user")==f.col("b.user")) & (f.col("b.value")==2),"leftouter")\
    .select("a.user",pref+"_cnt1","b.cnt").withColumnRenamed("cnt",pref+"_cnt2")
    DF2 = DF2.alias("a").join(DF.alias('b'),(f.col("a.user")==f.col("b.user")) & (f.col("b.value")==3),"leftouter")\
    .select("a.user",pref+"_cnt1",pref+"_cnt2","b.cnt").withColumnRenamed("cnt",pref+"_cnt3")
    DF2 = DF2.alias("a").join(DF.alias('b'),(f.col("a.user")==f.col("b.user")) & (f.col("b.value")==4),"leftouter")\
    .select("a.user",pref+"_cnt1",pref+"_cnt2",pref+"_cnt3","b.cnt").withColumnRenamed("cnt",pref+"_cnt4")
    DF2 = DF2.alias("a").join(DF.alias('b'),(f.col("a.user")==f.col("b.user")) & (f.col("b.value")==5),"leftouter")\
    .select("a.user",pref+"_cnt1",pref+"_cnt2",pref+"_cnt3",pref+"_cnt4","b.cnt").withColumnRenamed("cnt",pref+"_cnt5")
    DF2 = DF2.alias("a").join(DF.alias('b'),(f.col("a.user")==f.col("b.user")) & (f.col("b.value")==6),"leftouter")\
    .select("a.user",pref+"_cnt1",pref+"_cnt2",pref+"_cnt3",pref+"_cnt4",pref+"_cnt5","b.cnt").withColumnRenamed("cnt",pref+"_cnt6")
    DF2=DF2.fillna(0)
    return DF2

def link_outclass_intag(user,b):
    dd = [(str(user[i][0]), int(b[i])) for i in range(len(user))]
    df= sqlContext.createDataFrame(sc.parallelize(dd),schema=cSchema)
    return df

def link_outclass_intag_probs(user,b):
    dd = [(str(user[i][0]), float(b[i][0]),float(b[i][1]),float(b[i][2]),float(b[i][3]),float(b[i][4]),float(b[i][5])) for i in range(len(user))]
    df= sqlContext.createDataFrame(sc.parallelize(dd),schema=cSchema2)
    return df


def load_and_get_table_df(keys_space_name, table_name):
    table_df = sqlContext.read\
        .format("org.apache.spark.sql.cassandra")\
        .options(table=table_name, keyspace=keys_space_name)\
        .load()
    return table_df
    

### Import dataframe

In [5]:
# Class Definition
# 1: exchange
# 2: gambling
# 3: market
# 4: pool
# 5: mixer
# 6: service
# 7: wallet
# 8: lending

In [6]:
#########################################################################
#               IMPORT DATAFRAMES
#########################################################################

entity= sqlContext.read.parquet(path+path_directory+"/entity_feature_1.parquet")
address= sqlContext.read.parquet(path+path_directory+"/address_feature_1.parquet")
motif1 = sqlContext.read.parquet(path+path_directory+"/motifs1_1.parquet")
motif2 = sqlContext.read.parquet(path+path_directory+"/motifs2_1.parquet")

### Load Classifier

In [7]:
filename_rfc = modelpath+'/rfc_add.pkl'
loaded_model_rfc_add = pickle.load(open(filename_rfc, 'rb'))

filename_rfc = modelpath+'/rfc_mot1.pkl'
loaded_model_rfc_mot1 = pickle.load(open(filename_rfc, 'rb'))

filename_rfc = modelpath+'/rfc_mot2.pkl'
loaded_model_rfc_mot2 = pickle.load(open(filename_rfc, 'rb'))

filename_rfc = modelpath+'/final_gbc.pkl'
loaded_model_rfc_final = pickle.load(open(filename_rfc, 'rb'))

/usr/local/lib/python3.5/dist-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### Store DF into Cassandra

In [11]:
entity=entity.withColumnRenamed("count_recv","tx_recv")
entity=entity.withColumnRenamed("count_sent","tx_sent")
entity=entity.withColumnRenamed("balancerecv","amount_recv")
entity=entity.withColumnRenamed("balancesend","amount_sent")
entity=entity.drop("height")
entity.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table="entity", keyspace="kryptosare_test")\
.save()

In [8]:
####################################################################
#               ADDRESS ML
####################################################################

address_feature = address.fillna(0)
add_X=address.select("count_rec","totamount_rec","count_sent","totamount_sent","balance","unique","sibling")
#Round amount field
add_X = add_X.withColumn("totamount_rec", f.round(add_X["totamount_rec"], 6))
add_X = add_X.withColumn("totamount_sent", f.round(add_X["totamount_sent"], 6))
add_X = add_X.withColumn("balance", f.round(add_X["balance"], 6))

#Transform input/output dataframe in vector
X_add = add_X.collect()
#Reshape the output vector

In [12]:
probs=loaded_model_rfc_add.predict(X_add)
address_name=address.select("address").collect()
df_sp=link_outclass_intag(address_name,probs)
df_sp=df_sp.withColumnRenamed("user","address")
df_sp=df_sp.withColumnRenamed("label","class")
address_to_store=address.join(df_sp,"address")
address_to_store=address_to_store.drop("height")
address_to_store=address_to_store.withColumnRenamed("count_rec","tx_recv")
address_to_store=address_to_store.withColumnRenamed("count_sent","tx_sent")
address_to_store=address_to_store.withColumnRenamed("totamount_rec","amount_recv")
address_to_store=address_to_store.withColumnRenamed("totamount_sent","amount_sent")
address_to_store.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table="address", keyspace="kryptosare_test")\
.save()

In [13]:
# Format motifs amount data (round and cluster to reduce the amount of data)
motif11=motif1.withColumn("amount_recv",(f.round(f.col("amount_recv")/100000)*100000))
motif11=motif11.withColumn("amount_sent",(f.round(f.col("amount_sent")/100000)*100000))
motif11=motif11.withColumn("fees",(f.round(f.col("fees")/100)*100))
motif11=motif11.groupby("outuser","address_recv_dist","address_sent_dist","amount_recv","tx_sent","amount_sent","tx_recv_tot","fees","loop_in_out","direct_in_out")\
.agg(f.count("outuser").alias("repetition"))
df_motifs1=motif11.randomSplit([0.5,0.5])
df_motifs1[0].write.parquet(path+path_directory+"/motifs1_part1.parquet")
df_motifs1[1].write.parquet(path+path_directory+"/motifs1_part2.parquet")

In [18]:
import uuid
from pyspark.sql.functions import udf

In [17]:
for yy in range(1,3):

    df_motifs1_part=sqlContext.read.parquet(path+path_directory+"/motifs1_part"+str(yy)+".parquet")
    df_motifs1_part=df_motifs1_part.withColumn("id",f.monotonically_increasing_id())
    df_motifs1=df_motifs1_part.randomSplit([0.2,0.2,0.2,0.2,0.2])

    X_train_mot1=[]
    y_train_mot1=[]
    id_list=[]

    for i in range(0,5):
        #Split the input and the output from dataframe 
        mot1_X=df_motifs1[i].select("address_recv_dist","amount_recv","tx_sent","address_sent_dist","amount_sent","tx_recv_tot","fees","loop_in_out","direct_in_out","repetition")
        id_listX=df_motifs1[i].select("id")
        #Round amount field
        mot1_X = mot1_X.withColumn("amount_recv", f.round(mot1_X["amount_recv"], 6))
        mot1_X = mot1_X.withColumn("amount_sent", f.round(mot1_X["amount_sent"], 6))
        mot1_X = mot1_X.withColumn("fees", f.round(mot1_X["fees"], 6))

        #Transform input/output dataframe in vector
        X_train_mot1_ = mot1_X.collect()
        X_train_mot1 = X_train_mot1+X_train_mot1_
        id_list_=id_listX.collect()
        id_list=id_list+id_list_
        
    probs=loaded_model_rfc_mot1.predict(X_train_mot1)
    df_sp=link_outclass_intag(id_list,probs)
    df_sp=df_sp.withColumnRenamed("user","id")
    df_sp=df_sp.withColumnRenamed("label","class")
    motif11_to_store=df_motifs1_part.join(df_sp,"id")
    uuidUdf= udf(lambda : str(uuid.uuid4()),StringType())
    motif11_to_store_noid=motif11_to_store.drop("id")
    motif11_to_store_noid=motif11_to_store_noid.withColumn("id",uuidUdf())
    motif11_to_store_noid.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="motifs1", keyspace="kryptosare_test")\
    .save()


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [20]:
# Format motifs amount data (round and cluster to reduce the amount of data)

motif2=motif2.fillna(0,subset=["amount_sent_from_in","fee1"])
motif21=motif2.withColumn("amount_recv_to_out",(f.round(f.col("amount_recv_to_out")/1000000)*1000000))
motif21=motif21.withColumn("amount_sent_from_mid",(f.round(f.col("amount_sent_from_mid")/1000000)*1000000))
motif21=motif21.withColumn("amount_recv_to_mid",(f.round(f.col("amount_recv_to_mid")/1000000)*1000000))
motif21=motif21.withColumn("amount_sent_from_in",(f.round(f.col("amount_sent_from_in")/1000000)*1000000))

motif21=motif21.withColumn("fee2",(f.round(f.col("fee2")/1000)*1000))
motif21=motif21.withColumn("fee1",(f.round(f.col("fee1")/1000)*1000))
motif21=motif21.groupby("outuser","address_recv_dist_to_out","amount_recv_to_out","fee2",\
                             "tx_sent_from_mid","address_sent_from_mid","amount_sent_from_mid",\
                             "address_recv_to_mid","amount_recv_to_mid","tx_sent_from_in","address_sent_from_in",\
                             "amount_sent_from_in","fee1","loop_mid_out","loop_in_mid","loop_in_out",\
                             "direct_mid_out","direct_in_mid","direct_in_out")\
.agg(f.count("outuser").alias("repetition"))

motif21=motif21.withColumn("id",f.monotonically_increasing_id())
df_motifs2=motif21.randomSplit([0.2,0.2,0.2,0.2,0.2])

df_motifs2[0].write.parquet(path+path_directory+"/motifs2_part1.parquet")
df_motifs2[1].write.parquet(path+path_directory+"/motifs2_part2.parquet")
df_motifs2[2].write.parquet(path+path_directory+"/motifs2_part3.parquet")
df_motifs2[3].write.parquet(path+path_directory+"/motifs2_part4.parquet")
df_motifs2[4].write.parquet(path+path_directory+"/motifs2_part5.parquet")

In [21]:
for yy in range(1,6):
    df_motifs2_part=sqlContext.read.parquet(path+path_directory+"/motifs2_part"+str(yy)+".parquet")
    df_motifs2=df_motifs2_part.randomSplit([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])

    X_train_mot2=[]
    y_train_mot2=[]
    id_list=[]

    for i in range(0,10):
        #Split the input and the output from dataframe 
        mot2_X=df_motifs2[i].select("address_recv_dist_to_out","amount_recv_to_out","fee2",\
                                 "tx_sent_from_mid","address_sent_from_mid","amount_sent_from_mid",\
                                 "address_recv_to_mid","amount_recv_to_mid","tx_sent_from_in","address_sent_from_in",\
                                 "amount_sent_from_in","fee1","loop_mid_out","loop_in_mid","loop_in_out",\
                                 "direct_mid_out","direct_in_mid","direct_in_out","repetition")

        mot2_id=df_motifs2[i].select("id")

        #Round amount field
        mot2_X = mot2_X.withColumn("amount_recv_to_out", f.round(mot2_X["amount_recv_to_out"], 6))
        mot2_X = mot2_X.withColumn("amount_sent_from_mid", f.round(mot2_X["amount_sent_from_mid"], 6))
        mot2_X = mot2_X.withColumn("amount_recv_to_mid", f.round(mot2_X["amount_recv_to_mid"], 6))
        mot2_X = mot2_X.withColumn("amount_sent_from_in", f.round(mot2_X["amount_sent_from_in"], 6))
        mot2_X = mot2_X.withColumn("fee2", f.round(mot2_X["fee2"], 6))
        mot2_X = mot2_X.withColumn("fee1", f.round(mot2_X["fee1"], 6))

        #Transform input/output dataframe in vector
        X_train_mot2_ = mot2_X.collect()
        id_list_ = mot2_id.collect()

        X_train_mot2 = X_train_mot2+X_train_mot2_
        id_list = id_list+id_list_

    probs=loaded_model_rfc_mot2.predict(X_train_mot2)
    df_sp=link_outclass_intag(id_list,probs)
    df_sp=df_sp.withColumnRenamed("user","id")
    df_sp=df_sp.withColumnRenamed("label","class")
    motif21_to_store=motif21.join(df_sp,"id")

    uuidUdf= udf(lambda : str(uuid.uuid4()),StringType())
    motif21_to_store_noid=motif21_to_store.drop("id")
    motif21_to_store_noid=motif21_to_store_noid.withColumn("id",uuidUdf())

    motif21_to_store_noid.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="motifs2", keyspace="kryptosare_test")\
    .save()

### Cascading final

In [22]:
y_pred_rfc_add= load_and_get_table_df("kryptosare_test", "address")
y_pred_rfc_mot1= load_and_get_table_df("kryptosare_test", "motifs1")
y_pred_rfc_mot2= load_and_get_table_df("kryptosare_test", "motifs2")

In [23]:
y_pred_rfc_add_array=y_pred_rfc_add.groupBy("user","class").agg(f.countDistinct("address").alias("rep"))
y_pred_rfc_add_array=y_pred_rfc_add_array.withColumnRenamed("user","user2")

f1=y_pred_rfc_add_array.where(f.col("class")==1)
f2=y_pred_rfc_add_array.where(f.col("class")==2)
f3=y_pred_rfc_add_array.where(f.col("class")==3)
f4=y_pred_rfc_add_array.where(f.col("class")==4)
f5=y_pred_rfc_add_array.where(f.col("class")==5)
f6=y_pred_rfc_add_array.where(f.col("class")==6)

y_pred_rfc_add_ent=y_pred_rfc_add.groupBy("user").agg(f.first("user").alias("user2")).drop("user2")

In [24]:
first=y_pred_rfc_add_ent.alias("a").join(f1.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","add_cnt1")

first=first.alias("a").join(f2.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","add_cnt2")

first=first.alias("a").join(f3.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","add_cnt3")

first=first.alias("a").join(f4.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","add_cnt4")

first=first.alias("a").join(f5.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","add_cnt5")

y_end_add=first.alias("a").join(f6.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","add_cnt6")

y_end_add=y_end_add.fillna(0)

In [25]:
y_pred_rfc_mot1_array=y_pred_rfc_mot1.groupBy("outuser","class").agg(f.count("id").alias("rep"))
y_pred_rfc_mot1_array=y_pred_rfc_mot1_array.withColumnRenamed("outuser","user2")

f1=y_pred_rfc_mot1_array.where(f.col("class")==1)
f2=y_pred_rfc_mot1_array.where(f.col("class")==2)
f3=y_pred_rfc_mot1_array.where(f.col("class")==3)
f4=y_pred_rfc_mot1_array.where(f.col("class")==4)
f5=y_pred_rfc_mot1_array.where(f.col("class")==5)
f6=y_pred_rfc_mot1_array.where(f.col("class")==6)

y_pred_rfc_mot1_array_ent=y_pred_rfc_mot1.groupBy("outuser").agg(f.first("outuser").alias("user")).drop("outuser")

In [26]:
first=y_pred_rfc_mot1_array_ent.alias("a").join(f1.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot1_cnt1")

first=first.alias("a").join(f2.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot1_cnt2")

first=first.alias("a").join(f3.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot1_cnt3")

first=first.alias("a").join(f4.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot1_cnt4")

first=first.alias("a").join(f5.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot1_cnt5")

y_end_mot1=first.alias("a").join(f6.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot1_cnt6")

y_end_mot1=y_end_mot1.fillna(0)

In [27]:
y_pred_rfc_mot2_array=y_pred_rfc_mot2.groupBy("outuser","class").agg(f.count("id").alias("rep"))
y_pred_rfc_mot2_array=y_pred_rfc_mot2_array.withColumnRenamed("outuser","user2")

f1=y_pred_rfc_mot2_array.where(f.col("class")==1)
f2=y_pred_rfc_mot2_array.where(f.col("class")==2)
f3=y_pred_rfc_mot2_array.where(f.col("class")==3)
f4=y_pred_rfc_mot2_array.where(f.col("class")==4)
f5=y_pred_rfc_mot2_array.where(f.col("class")==5)
f6=y_pred_rfc_mot2_array.where(f.col("class")==6)

y_pred_rfc_mot2_array_ent=y_pred_rfc_mot2.groupBy("outuser").agg(f.first("outuser").alias("user")).drop("outuser")

In [28]:
first=y_pred_rfc_mot2_array_ent.alias("a").join(f1.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot2_cnt1")

first=first.alias("a").join(f2.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot2_cnt2")

first=first.alias("a").join(f3.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot2_cnt3")

first=first.alias("a").join(f4.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot2_cnt4")

first=first.alias("a").join(f5.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot2_cnt5")

y_end_mot2=first.alias("a").join(f6.alias("b"),f.col("a.user")==f.col("b.user2"),"leftouter").drop("user2","class")\
.withColumnRenamed("rep","mot2_cnt6")

y_end_mot2=y_end_mot2.fillna(0)

In [29]:
#Normalize the information of the prediction in each dataframe
col=["add_cnt1","add_cnt2","add_cnt3","add_cnt4","add_cnt5","add_cnt6"]
   
y_end_add = y_end_add.withColumn("sum",sum([y_end_add[c] for c in col]))
for c in col:
    y_end_add = y_end_add.withColumn(c,f.round(f.col(c)/f.col("sum")*100,2))
       
########################################### 
col=["mot1_cnt1","mot1_cnt2","mot1_cnt3","mot1_cnt4","mot1_cnt5","mot1_cnt6"]
    
y_end_mot1= y_end_mot1.withColumn("sum",sum([y_end_mot1[c] for c in col]))
for c in col:
    y_end_mot1 = y_end_mot1.withColumn(c,f.round(f.col(c)/f.col("sum")*100,2))
    
###########################################   
col=["mot2_cnt1","mot2_cnt2","mot2_cnt3","mot2_cnt4","mot2_cnt5","mot2_cnt6"]
y_end_mot2 = y_end_mot2.withColumn("sum",sum([y_end_mot2[c] for c in col]))
for c in col:
    y_end_mot2 = y_end_mot2.withColumn(c,f.round(f.col(c)/f.col("sum")*100,2))

In [30]:
y_pred_rfc_add=y_end_add.drop("sum")
y_pred_rfc_mot1=y_end_mot1.drop("sum")
y_pred_rfc_mot2=y_end_mot2.drop("sum")

In [31]:
#Join all the dataframe information into one
df_final_rf = entity.join(y_pred_rfc_add,['user'],"outer")
df_final_rf = df_final_rf.join(y_pred_rfc_mot1,['user'],"outer")
df_final_rf = df_final_rf.join(y_pred_rfc_mot2,['user'],"outer")
df_final_rf = df_final_rf.fillna(0)

In [34]:
#Create final dataframe, split into X and y and convert them into list
rf_final_X=df_final_rf.select("amount_recv","amount_sent","balance","tx_recv","tx_sent","add_recv","add_sent",\
                        "add_cnt1","add_cnt2","add_cnt3","add_cnt4","add_cnt5","add_cnt6",\
                        "mot1_cnt1","mot1_cnt2","mot1_cnt3","mot1_cnt4","mot1_cnt5","mot1_cnt6",\
                        "mot2_cnt1","mot2_cnt2","mot2_cnt3","mot2_cnt4","mot2_cnt5","mot2_cnt6")

#Round amount field
rf_final_X = rf_final_X.withColumn("amount_recv", f.round(rf_final_X["amount_recv"], 6))
rf_final_X = rf_final_X.withColumn("amount_sent", f.round(rf_final_X["amount_sent"], 6))
rf_final_X = rf_final_X.withColumn("balance", f.round(rf_final_X["balance"], 6))

#Transform input/output dataframe in vector
rf_final_X_full= rf_final_X.fillna(0).collect()


In [35]:
probs=loaded_model_rfc_final.predict_proba(rf_final_X_full)
entityname=df_final_rf.select("user").collect()
df_sp=link_outclass_intag_probs(entityname,probs)
entity_to_store=df_final_rf.join(df_sp,"user")
entity_to_store=entity_to_store.drop("height")

entity_to_store.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table="enriched_entity", keyspace="kryptosare_test")\
.save()